In [1]:
import neat
import numpy as np

filename = 'A:/Профиль/Rab Table/Учёба/2/нейроэволюционные/Методические/data_to_use/cancer1 copy.dt'
raw_data = np.loadtxt(filename)

In [2]:
print(raw_data)

[[0.2 0.1 0.1 ... 0.1 1.  0. ]
 [0.2 0.1 0.1 ... 0.1 1.  0. ]
 [0.5 0.1 0.1 ... 0.1 1.  0. ]
 ...
 [0.6 1.  1.  ... 0.7 0.  1. ]
 [0.5 0.7 1.  ... 0.1 0.  1. ]
 [0.1 0.1 0.1 ... 0.1 1.  0. ]]


In [19]:
# make x and y from x
y = raw_data[:, 10] # [: , 9] - это значит, что мы берем все строки, но только столбец с индексом 9
x = raw_data[:, 0:9]
y = y.astype(float)

# make a list 'inputs' of sets from x like this: [(0.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 1.0)]
inputs = []
for i in range(len(x)):
    inputs.append(tuple(x[i]))

# make a list 'outputs' of sets from y like this: [(0.0,), (1.0,), (1.0,), (0.0,)]
outputs = []
for i in range(len(y)):
    outputs.append(tuple([y[i]]))

print(outputs[0])
print(inputs[0])



(0.0,)
(0.2, 0.1, 0.1, 0.1, 0.2, 0.1, 0.2, 0.1, 0.1)


In [20]:
# split x and y into train, validation, test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.75)

In [21]:
print(x_train[1])
print(y_train[1])
print(x_test[1])
print(y_test[1])

(0.4, 0.4, 0.2, 0.1, 0.2, 0.5, 0.2, 0.1, 0.2)
(0.0,)
(0.1, 0.1, 0.1, 0.1, 0.5, 0.1, 0.3, 0.1, 0.1)
(0.0,)


In [22]:
import pickle
import graphviz

In [23]:
len(x_train)

174

In [24]:
import my_visualize as visualize
import matplotlib.pyplot as plt

In [25]:
len(x_train)

174

In [ ]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = len(x_train)
        for xi, xo in zip(x_train, y_train):
            output = net.activate(xi)
            output = tuple(output)
            try:
                genome.fitness -= (output[0] - xo[0]) ** 2
                #print(output[0], ' -- ', xo[0], ' -- ', (output[0] - xo[0]) ** 2, genome.fitness, len(x_train))
            except Exception as e:
                print(output)
                print(xo)
                print(xi)
                print(e)
                raise e
        debug = 171.5
        if (debug < genome.fitness):
            debug = genome.fitness
            print(genome.fitness, len(x_train))
        
config_path = 'A:/Профиль/Rab Table/Учёба/2/Neuroevolutionary-computing/Код/config-cancer'
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)
population = neat.Population(config)
#population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)
#population.add_reporter(neat.Checkpointer(5))
winner = population.run(eval_genomes)
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

correct = 0
for xi, xo in zip(x_test, y_test):
    output = winner_net.activate(xi)
    if np.argmax(output) == xo:
        correct += 1

print('Accuracy:', correct / len(y_test))

with open('winner-mnist', 'wb') as f:
    pickle.dump(winner, f)

In [ ]:
node_names = {-1: 'x0',
               -2: 'x1',
                -3: 'x2',
                -4: 'x3',
                -5: 'x4',
                -6: 'x5',
                -7: 'x6',
                -8: 'x7',
                -9: 'x8',
                 0: 'OUTPUT'}

visualize.draw_net(config, winner, True, node_names=node_names, filename="winner-mnist.gv")

In [ ]:
visualize.plot_stats(stats, ylog=False, view=True)


In [ ]:
visualize.plot_species(stats, view=True)

# Hidden

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (200,200)
graph = graphviz.Source.from_file('winner-mnist.gv')
graph
